In [109]:
token = "Y9czvVBhfFYY1KG7GzBhtwOm2whsjGcHQ4bBHoPn32_EGGrGL0GvaEKV5Kx3z7Pbo-HehPQlN4jOdnPX7k0anA"
client_id = "Vs4Q7Xf-E_zmCdp_50k08utL9NDFHATRV1NTSQiW7FJPBH2XKnQvMvIyOBJERyav"
access_token = "HxR7zXc0wRInUmtPm61JS4fpyNnqDUq_Fc8uw_fjfJJ8aKa6TLjHluT08QXNvP8r"

In [110]:
!pip install lyricsgenius
!pip install openpyxl

In [111]:
import tensorflow as tf

import os
import json
from lyricsgenius import Genius

from keras.layers import Embedding, LSTM, Dense, Dropout, Activation, Bidirectional
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from tensorflow.compat.v1.keras.utils import to_categorical

import numpy as np
import pandas as pd

In [112]:
# used only for loading rock data

def load_rock_data():
    df = pd.read_excel("./selected_songs.xlsx")
    lyrics = "\n".join(df["lyrics"]).lower().split("\n")
    del df
    
    return lyrics, False

# used only for loading beyonce data

def load_beyonce():
    if not os.path.exists("./Lyrics_Beyoncé.json"):
        genius = Genius(access_token, skip_non_songs=True, 
                        excluded_terms=["(Remix)", "(Live)"],
                        remove_section_headers=True, timeout=60)
        artist = genius.search_artist('Beyoncé')
        artist.save_lyrics()

    with open("./Lyrics_Beyoncé.json", "r") as f:
        text = f.read()
        data = json.loads(text)

    lyrics = [] 
    for i in range(len(data["songs"])):
        for sentence in data["songs"][i]["lyrics"].split("\n"):
            lyrics.append(sentence.lower())
            
    return lyrics, True

In [113]:
# change this line depending on which model you are using
lyrics, is_beyonce = load_rock_data()

# removing duplicate lines while keeping same order
corpus = []
text_set = set()

for line in lyrics:
    if line not in text_set:
        corpus.append(line)
        text_set.add(line)

del text_set

tokenizer = Tokenizer()
tokenizer.fit_on_texts(corpus)
total_words = len(tokenizer.word_index) + 1

# create sequences of texts. if sequence are length n, then for each element in the sequence
# the 0 - (n-1) elements are used to predict the nth element

inputs = []
for line in corpus:
    tokens = tokenizer.texts_to_sequences([line])[0]
    
    for i in range(1, len(tokens)):
        seq = tokens[:i+1]

        # keep only actual lyrics to beyonce songs (not her speeches, interviews, etc) by restricing length.
        # This is not necessary for the rock songs dataframe.
        if is_beyonce:
            if len(seq) < 25:
                inputs.append(seq)
        else:
            inputs.append(seq)
        
# pad sequences to all be the length of the longest line
max_len = max([len(x) for x in inputs])
inputs = np.array(pad_sequences(inputs,
                       maxlen = max_len, padding='pre'))

# pad sequences to all be the length of the longest line

max_len = max([len(x) for x in inputs])
inputs = np.array(pad_sequences(inputs,
                       maxlen = max_len, padding='pre'))

# construct x and y data

X_data, y_data = inputs[:, :-1], inputs[:, -1]
y_data = to_categorical(y_data, num_classes=total_words)

In [114]:
# construct model
model = Sequential()
model.add(Embedding(total_words, 80, input_length=max_len-1))
# Add an LSTM Layer
model.add(Bidirectional(LSTM(80, return_sequences=True)))  
# A dropout layer for regularisation
model.add(Dropout(0.2))
# Add another LSTM Layer
model.add(LSTM(80))
model.add(Dropout(0.2))
model.add(Dense(100, activation='relu'))
# In the last layer, the shape should be equal to the total number of words present in our corpus
model.add(Dense(total_words, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics='accuracy')  #(# Pick a loss function and an optimizer)
print(model.summary())


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 451, 80)           761520    
                                                                 
 bidirectional_2 (Bidirectio  (None, 451, 160)         103040    
 nal)                                                            
                                                                 
 dropout_4 (Dropout)         (None, 451, 160)          0         
                                                                 
 lstm_5 (LSTM)               (None, 80)                77120     
                                                                 
 dropout_5 (Dropout)         (None, 80)                0         
                                                                 
 dense_4 (Dense)             (None, 100)               8100      
                                                      

In [115]:
import os

# load 150 epoch rock model. Change path name to load other models (i.e. beyonce). 
# Whenever you change models, make sure to rerun all all previous cells.

if not os.path.exists("./rock_modelv5.h5"):
    history = model.fit(X_data, y_data, epochs= 150, verbose=1, validation_split=0.2)
    model.save("./rock_modelv5.h5")
else:
    model = tf.keras.models.load_model("./rock_modelv5.h5")
    


In [119]:
def generate(seed_text, num_words):
    for i in range(num_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list],
                    maxlen=max_len-1, padding='pre')
        predicted = model.predict(token_list, verbose=0).argmax()
        output_word = ""

        for word, index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        seed_text += " " + output_word
    print(seed_text)

In [120]:
generate("once i was", 4)
generate("never", 9)
generate("once you", 6)

once i was born to be happy
never stop 'til the sun is falling in the clouds
once you were the one collecting ideas oooh
